In [1]:
# import Glop linear solver package
from ortools.linear_solver import pywraplp as glp

In [2]:
# model parameters
project = ['Painting', 'Drawing', 'Nature craft', 'Rhythm band', 'Relay races', 'Basketball', 'Internet', 'Creative writing', 'Games']
time = [30,20,30,20,45,60,45,30,40]
campersPop = [4,5,3,5,2,1,1,4,1]
counselorsPop = [2,2,1,5,1,3,1,3,2]


In [3]:
# initialize model object
mymodel = glp.Solver('Capital Budget', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)



In [4]:
# create decision variables
select = list(range(len(project)))
for j in range(len(project)):
    select[j] = mymodel.IntVar(0,1, project[j])
    
print('select = ', select)

select =  [Painting, Drawing, Nature craft, Rhythm band, Relay races, Basketball, Internet, Creative writing, Games]


In [5]:
# create objective function
tot_npv = mymodel.Objective()
tot_npv.SetMaximization()
for j in range(len(project)):
    tot_npv.SetCoefficient(select[j], campersPop[j]+counselorsPop[j])

#display objective function
for j in range(len(project)):
    print(select[j], tot_npv.GetCoefficient(select[j]))

Painting 6.0
Drawing 7.0
Nature craft 4.0
Rhythm band 10.0
Relay races 3.0
Basketball 4.0
Internet 2.0
Creative writing 7.0
Games 3.0


In [6]:
# create constraints
constr = list()
#a. At most one art activity can be done.
constr.append(mymodel.Constraint(0,1))
constr[0].SetCoefficient(select[0], 1)
constr[0].SetCoefficient(select[1], 1)
constr[0].SetCoefficient(select[2], 1)
    

In [7]:
#b. No more than two computer activities can be done.
constr.append(mymodel.Constraint(0,2))
constr[1].SetCoefficient(select[6], 1)
constr[1].SetCoefficient(select[7], 1)
constr[1].SetCoefficient(select[8], 1)
    

In [8]:
#c. If basketball is chosen, then rhythm band must be chosen.
constr.append(mymodel.Constraint(0,0))
constr[2].SetCoefficient(select[3], -1)
constr[2].SetCoefficient(select[5], 1)


In [9]:
#d. At least 120 minutes of activities must be selected
#e. No more than 165 minutes of activities may be selected.
constr.append(mymodel.Constraint(120,165))
for i in range(len(project)):
    constr[3].SetCoefficient(select[i], time[i])


In [10]:
#f. To keep the staff happy, the counselor rating should be no higher than 10.
constr.append(mymodel.Constraint(0,10))
for i in range(len(project)):
    constr[4].SetCoefficient(select[i], counselorsPop[i])
    

In [11]:
# Solve the model and print optimal solution
status = mymodel.Solve()                 # solve mymodel and display the solution

print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Optimal Value = %.2f' % tot_npv.Value())

# Display optimal solution
for j in range(len(project)):
    print('%s %4.2f' % (project[j], select[j].solution_value()))

Solution Status = 0
Number of variables = 9
Number of constraints = 5
Optimal Solution:
Optimal Value = 21.00
Painting 0.00
Drawing 0.00
Nature craft 1.00
Rhythm band 1.00
Relay races 1.00
Basketball 1.00
Internet 0.00
Creative writing 0.00
Games 0.00


In [12]:
# store coefficients and RHS for cut constraint
# -select[0] - select[1] - select[2] + select[3] + select[4] <=2-1
# 0-1-0+1+1<=1
a = list()
M = 0

for i in range(len(select)):
    if select[i].solution_value() == 1:
        a.append(1)
        M+=1
    elif select[i].solution_value() == 0:
        a.append(-1)
print(M, a)

4 [-1, -1, 1, 1, 1, 1, -1, -1, -1]


In [14]:
#add cut constraint
cut_constr = mymodel.Constraint(-mymodel.infinity(), M-1)
for i in range(len(project)):
    cut_constr.SetCoefficient(select[i], a[i])

    
#re-optimize and display next solution
mymodel.Solve()
print('Optimal Value =', tot_npv.Value())
i = 0
for i in range(len(project)):
    if select[i].solution_value() == 1:
            print('%s %4.2f' % (project[i], select[i].solution_value()))
        
# relax cut constraint
cut_constr.SetBounds(-mymodel.infinity(),mymodel.infinity())

Optimal Value = 20.0
Drawing 1.00
Relay races 1.00
Creative writing 1.00
Games 1.00
